In [2]:
# Imports
import random

import math

class Vertex:
    def __init__(self, key, x, y):
        self.key = key
        self.x = x
        self.y = y
        self.d = float('-inf')  # Distance from source
        self.h = None  # Heuristic distance to destination
        self.pi = None
        self.pi_set = set()
        self.priority = 0
        self.parent = None

    def __lt__(self, other):
        return self.priority < other.priority

    def __gt__(self, other):
        return self.priority > other.priority


class Graph:
    def __init__(self):
        self.vertices = {}
        self.adjacency_list = {}

    def add_vertex(self, key, x, y):
        if key not in self.vertices:
            self.vertices[key] = Vertex(key, x, y)
            self.adjacency_list[key] = []

    def add_edge(self, u, v):
        if u in self.vertices and v in self.vertices and u != v:
            if v not in self.adjacency_list[u]:
                self.adjacency_list[u].append(v)
            if u not in self.adjacency_list[v]:
                self.adjacency_list[v].append(u)  # For undirected graph


    def get_adjacency_list(self, vertex_key):
        if vertex_key in self.vertices:
            return self.adjacency_list[vertex_key]
        else:
            return []
    
    def max_degree(self):
        max_degree = 0
        for vertex_key in self.vertices.keys():
            degree = len(self.adjacency_list[vertex_key])
            max_degree = max(max_degree, degree)
        return max_degree
    
    def average_degree(self):
        total_degree = 0
        for vertex_key in self.vertices.keys():
            total_degree += len(self.adjacency_list[vertex_key])
        if len(self.vertices) > 0:
            return total_degree / len(self.vertices)
        else:
            return 0

    def generate_geometric_graph(self, n, r):

        for key in range(n):
            self.add_vertex(key,random.uniform(0, 1), random.uniform(0, 1))

        for u in self.vertices:
            for v in self.vertices:
                if u != v and euclidean_distance(self.vertices[u], self.vertices[v]) <= r:
                    self.adjacency_list[u].append(v)
                    self.adjacency_list[v].append(u)


def euclidean_distance(v1, v2):
    return math.sqrt(((v1.x - v2.x) ** 2) + ((v1.y - v2.y) ** 2))



n_values = [300, 400, 500]
min_ratios = [0.9, 0.8, 0.7]
max_ratios = [0.95, 0.9, 0.8]

graph = Graph()

def largest_connected_component(graph):
    def dfs(graph, visited, vertex, component):
        visited.add(vertex)
        component.append(vertex)
        for neighbor in graph.get_adjacency_list(vertex):
            if neighbor not in visited:
                dfs(graph, visited, neighbor, component)

    visited = set()
    largest_component = []
    for vertex in graph.vertices:
        if vertex not in visited:
            component = []
            dfs(graph, visited, vertex, component)
            if len(component) > len(largest_component):
                largest_component = component

    lcc_graph = Graph()
    for vertex_key in largest_component:
        vertex = graph.vertices[vertex_key]
        lcc_graph.add_vertex(vertex.key, vertex.x, vertex.y)
        for neighbor_key in graph.get_adjacency_list(vertex_key):
            if neighbor_key in largest_component:
                lcc_graph.add_edge(vertex.key, neighbor_key)

    return lcc_graph


def binary_search_r(n, min_ratio, max_ratio):
    low = 0.0
    high = 1.0
    while low <= high:
        graph = Graph()
        mid = (low + high) / 2
        vlcc_min = int(min_ratio * n)
        vlcc_max = int(max_ratio * n)
        graph.generate_geometric_graph(n, mid)
        lcc = largest_connected_component(graph)
        lcc_size = len(lcc.vertices)
        if vlcc_min <= lcc_size <= vlcc_max:
            return mid, lcc
        elif lcc_size > vlcc_min:
            high = mid - 0.00001
        else:
            low = mid + 0.00001
    return None, None  # No suitable r found


def store_graph_to_file(graph, filename):
    with open(filename, 'w') as f:
        processed_edges = set()  # To avoid duplicates
        for u in graph.vertices:
            for v in graph.get_adjacency_list(u):
                # Generate a unique key for the edge (u, v) by sorting the vertices' keys
                edge_key = (u, v) if u < v else (v, u)
                if edge_key not in processed_edges:
                    vertex_u = graph.vertices[u]
                    vertex_v = graph.vertices[v]
                    f.write(f"{u} {vertex_u.x} {vertex_u.y} {v} {vertex_v.x} {vertex_v.y}\n")
                    processed_edges.add(edge_key)


n_values = [300, 400, 500]
min_ratios = [0.9, 0.8, 0.7]
max_ratios = [0.95, 0.9, 0.8]

# for n, min_ratio, max_ratio in zip(n_values, min_ratios, max_ratios):
#     r, lcc = binary_search_r(n, min_ratio, max_ratio)
#     if r is not None:
#         store_graph_to_file(lcc, f"graph_n{n}_r{r}.EDGES")
#         print(f"Graph with n={n} and r={r} stored successfully.")
#     else:
#         print(f"No suitable r found for n={n}.")


def read_graph_from_file(filename):
    graph = Graph()
    with open(filename, 'r') as f:
        for line in f:
            data = line.split()
            u = int(data[0])
            x1 = float(data[1])
            y1 = float(data[2])
            v = int(data[3])
            x2 = float(data[4])
            y2 = float(data[5])

            if u not in graph.vertices:
                graph.add_vertex(u, x1, y1)
            if v not in graph.vertices:
                graph.add_vertex(v, x2, y2)

            graph.add_edge(u, v)

    return graph


graph = read_graph_from_file("graph_n400_r0.06249125.EDGES")

if 13 in graph.vertices:
    print("yes")


yes


In [ ]:
# import networkx as nx
# import matplotlib.pyplot as plt


# def draw_graph(edges=None, cordinates=None, adj_list=None):
#     G = nx.DiGraph()
#     if adj_list:
#         G.add_nodes_from(adj_list.keys())
#         for k, v in adj_list.items():
#             G.add_edges_from(([(k, t) for t in v]))
#         nx.draw(G, with_labels=True)
#         plt.show()
#     else:
#         G.add_nodes_from(cordinates.keys())
#         nx.set_node_attributes(G, cordinates, 'pos')
#         adjacency_list = {}
#         for u, v in edges:
#             adjacency_list.setdefault(u, []).append(v)
#             adjacency_list.setdefault(v, []).append(u)
#         for k, v in adjacency_list.items():
#             G.add_edges_from(([(k, t) for t in v]))
#         _, ax = plt.subplots(figsize=(10, 5))
#         nx.draw(G, pos=nx.get_node_attributes(G, 'pos'), with_labels=True, ax=ax)
#         ax.set_axis_on()
#         ax.tick_params(left=True, bottom=True, labelleft=True, labelbottom=True)

# # Generate a random geometric graph
# n = 20  # Number of vertices
# r = 0.3  # Maximum distance for edge creation
# graph = Graph()

# graph.generate_geometric_graph(n, r)
# draw_graph(adj_list=graph.adjacency_list)

# lcc = largest_connected_component(graph)
# draw_graph(adj_list=lcc.adjacency_list)


In [3]:
#  A star
import heapq

def initialize_single_source_max(graph, source, destination):
    for vertex in graph.vertices.values():
        vertex.d = float('-inf')
        vertex.h = euclidean_distance(vertex, destination)
        vertex.pi = None
        vertex.pi_set = set()

    source.d = 0

def relax_max(u, v):
    if v.d < (u.d + 1):
        v.d = u.d + 1
        v.pi = u
        return True
    return False

def a_star_max(graph, source, destination):
    initialize_single_source_max(graph, source, destination)
    S = set()  # Closed list
    Q = []  # Max heap
    for vertex in graph.vertices.values():
        vertex.priority = vertex.d + vertex.h
        heapq.heappush(Q, vertex)
    heapq._heapify_max(Q)

    while Q:
        u = heapq._heappop_max(Q)
        S.add(u)

        for v in graph.get_adjacency_list(u.key):
            v = graph.vertices[v]
            if v in u.pi_set:
                continue
            if relax_max(u, v):
                v.pi_set = u.pi_set | {u}
                if v in S:
                    S.remove(v)
                    heapq.heappush(Q, vertex)
                    heapq._heapify_max(Q)
                else:
                    v.priority = v.d + v.h
                    heapq._heapify_max(Q)

final_path = []

for i in range(0, len(graph.vertices)):
    if i in graph.vertices:
        for j in range(0, len(graph.vertices)):
            if i != j and j in graph.vertices:
                s = graph.vertices[i]
                d = graph.vertices[j]
                a_star_max(graph, s, d)
                path = []
                x = d
                if x.d != float('-inf'):
                    visited = {x}
                    while x != s:
                        path.insert(0, x.key)
                        visited.add(x)
                        x = x.pi
                        if x in visited:
                            break
                    path.insert(0, s.key)
                    if (len(path) > len(final_path)):
                        final_path = path

final_path

                

[155,
 366,
 243,
 269,
 305,
 161,
 179,
 188,
 345,
 210,
 397,
 38,
 223,
 399,
 323,
 384,
 274,
 12,
 26,
 390,
 250,
 169,
 348,
 218,
 189,
 0,
 322,
 45,
 94,
 369,
 253,
 247,
 4,
 198,
 316,
 67,
 89,
 142,
 382,
 364,
 69,
 254,
 66,
 125,
 320,
 373,
 202,
 137,
 36,
 314,
 358,
 19,
 126,
 102,
 277,
 350,
 241,
 50,
 162,
 58,
 265,
 229,
 263,
 299,
 260,
 303,
 352,
 176,
 270,
 211,
 248,
 146,
 54,
 207,
 383,
 47,
 159,
 238,
 141,
 171,
 20,
 112,
 339,
 11,
 341,
 65,
 395,
 124,
 257,
 347,
 88,
 107,
 68,
 57,
 177,
 74,
 335,
 354,
 91,
 40,
 92,
 85,
 398,
 381,
 237,
 101,
 127,
 183,
 204,
 392,
 340,
 296,
 150,
 309,
 246,
 222,
 123,
 73,
 338,
 301,
 220,
 192,
 64,
 97,
 143,
 186,
 113,
 62,
 387,
 7,
 105,
 131,
 306,
 325,
 37,
 297,
 51,
 266,
 80,
 46,
 163,
 119,
 283,
 63,
 396,
 33,
 29,
 285,
 288,
 13,
 379,
 39,
 160,
 172,
 294,
 225,
 77,
 99,
 377,
 319]

In [ ]:
# def initialize_single_source(G, s):
#     for v in G.vertices.values():
#         v.d = float('-inf')
#     s.d = 0

# def relax(u, v):
#     # Modify relaxation to maximize distance
#     if v.d < u.d + 1:  # Assuming each edge has a weight of 1
#         v.d = u.d + 1
#         v.parent = u

# def Dijkstra(G, s):
#     initialize_single_source(G, s)
#     closed_set = set()
#     open_list = [(0, s)]  # Min heap based on distance
#     heapq.heapify(open_list)

#     while open_list:
#         dist, u = heapq.heappop(open_list)
#         if u in closed_set:
#             continue
#         closed_set.add(u)
#         for v_key in G.get_adjacency_list(u.key):
#             v = G.vertices[v_key]
#             if v not in closed_set:
#                 relax(u, v)
#                 heapq.heappush(open_list, (-v.d, v))  # Negate distance for max heap

#     # Construct the longest path by backtracking from the destination vertex
#     longest_path = []
#     current = max(G.vertices.values(), key=lambda v: v.d)
#     while current is not None:
#         longest_path.insert(0, current.key)
#         current = current.parent

#     return longest_path




# print(Dijkstra(graph, graph.vertices[6]))


# # for i in range(0, len(graph.vertices)):
# #     print(Dijkstra(graph, graph.vertices[i]))

# # final_path = []
# # for i in range(0, len(graph.vertices)):
# #     path = Dijkstra(graph, graph.vertices[i])
# #     if (len(path) > len(final_path)):
# #         final_path = path

# # final_path

In [ ]:
def DFSAlgo(graph):
    def dfs(node, visited, path):
        nonlocal max_length, longest_path
        visited[node] = True
        path.append(node)

        for neighbor in graph.adjacency_list[node]:
            if not visited[neighbor]:
                dfs(neighbor, visited, path)

        if len(path) > max_length:
            max_length = len(path)
            longest_path = path.copy()

        path.pop()
        visited[node] = False

    max_length = float('-inf')
    longest_path = []

    visited = {v: False for v in graph.vertices}

    for vertex in graph.vertices:
        dfs(vertex, visited, [])

    return longest_path


DFSAlgo(graph)

In [ ]:
# TESTING DFS - with save

def DFSAlgo(graph):
    max_length = float('-inf')
    longest_path = []

    def dfs(node, visited, path):
        nonlocal max_length, longest_path
        visited.add(node)
        path.append(node)

        if len(path) > max_length:
            max_length = len(path)
            longest_path = path.copy()

        for neighbor in graph.adjacency_list[node]:
            if neighbor not in visited:
                dfs(neighbor, visited, path)

        path.pop()
        visited.remove(node)

    visited = set()
    for vertex in graph.vertices:
        dfs(vertex, visited, [])

    return longest_path

DFSAlgo(graph)

In [ ]:
import threading
import multiprocessing
import numpy as np

def dfs(node, visited, path, max_length, longest_path):
    visited.add(node)
    path.append(node)

    if len(path) > max_length[0]:
        max_length[0] = len(path)
        longest_path[0] = path.copy()

    for neighbor in graph.adjacency_list[node]:
        if neighbor not in visited:
            dfs(neighbor, visited, path, max_length, longest_path)

    path.pop()
    visited.remove(node)

def dfs_thread(vertex, max_length, longest_path):
    visited = set()
    dfs(vertex, visited, [], max_length, longest_path)

def dfs_process(vertices, max_length, longest_path):
    threads = []
    for vertex in vertices:
        thread = threading.Thread(target=dfs_thread, args=(vertex, max_length, longest_path))
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()

def DFSAlgo(graph):
    max_length = [float('-inf')]
    longest_path = [[]]

    graph_vertices_array = np.array(graph.vertices).reshape(-1)
    chunks = np.array_split(graph_vertices_array, 4)

    with multiprocessing.Pool() as pool:
        pool.starmap(dfs_process, [(chunk, max_length, longest_path) for chunk in chunks])

    return longest_path[0]

# graph = ...  # Define your graph here
DFSAlgo(graph)

In [ ]:
from collections import deque

def BFSLongestPath(graph):
    def bfs(node):
        nonlocal max_length, longest_path
        queue = deque([(node, [node])])

        while queue:
            current_node, path = queue.popleft()
            for neighbor in graph.adjacency_list[current_node]:
                if neighbor not in path:
                    new_path = path + [neighbor]
                    queue.append((neighbor, new_path))
                    if len(new_path) > max_length:
                        max_length = len(new_path)
                        longest_path = new_path

    max_length = float('-inf')
    longest_path = []

    for vertex in graph.vertices:
        bfs(vertex)

    return longest_path

DFSAlgo(graph)


In [ ]:
# import heapq

# def dijkstra_longest_path(graph, start):
#     distances = {vertex: float('-inf') for vertex in graph.vertices}
#     distances[start] = 0

#     pq = [(0, start)]  # Priority queue (distance, vertex)
#     visited = set()

#     while pq:
#         distance, node = heapq.heappop(pq)

#         if node in visited:
#             continue

#         visited.add(node)

#         for neighbor in graph.adjacency_list[node]:
#             if neighbor not in visited:
#                 new_distance = distances[node] + 1  # Assuming unweighted graph
#                 if new_distance > distances[neighbor]:
#                     distances[neighbor] = new_distance
#                     heapq.heappush(pq, (-new_distance, neighbor))

#     return distances

# def longest_path_dijkstra(graph):
#     max_length = float('-inf')
#     longest_path = []

#     final_distance = None
#     for vertex in graph.vertices:
#         distances = dijkstra_longest_path(graph, vertex)
#         longest_distance = max(distances.values())
#         if longest_distance > max_length:
#             max_length = longest_distance
#             final_distance = distances
#             longest_path = [k for k, v in distances.items() if v == longest_distance]

#     return longest_path, final_distance

# # Example usage:
# # Replace Graph with your graph implementation
# # graph = Graph()  # Assuming you have a graph instance
# longest_path, final_distance = longest_path_dijkstra(graph)
# print("Longest Path:", longest_path, final_distance)

In [ ]:
import heapq

def dijkstra_longest_path(graph, start):
    distances = {vertex: float('-inf') for vertex in graph.vertices}
    distances[start] = 0
    previous = {vertex: None for vertex in graph.vertices}

    pq = [(0, start)]  # Priority queue (distance, vertex)
    visited = set()

    while pq:
        distance, node = heapq.heappop(pq)

        if node in visited:
            continue

        visited.add(node)

        for neighbor in graph.adjacency_list[node]:
            if neighbor not in visited:
                new_distance = distances[node] + 1  # Assuming unweighted graph
                if new_distance > distances[neighbor]:
                    distances[neighbor] = new_distance
                    previous[neighbor] = node
                    heapq.heappush(pq, (-new_distance, neighbor))

    return distances, previous

def construct_path(previous, start, end):
    path = []
    current = end
    while current is not None:
        path.append(current)
        current = previous[current]
    path.reverse()
    return path

def longest_path_dijkstra(graph):
    max_length = float('-inf')
    longest_path = []

    for vertex in graph.vertices:
        distances, previous = dijkstra_longest_path(graph, vertex)
        longest_distance = max(distances.values())
        if longest_distance > max_length:
            max_length = longest_distance
            end_vertex = max(distances, key=distances.get)
            longest_path = construct_path(previous, vertex, end_vertex)

    return longest_path, max_length

# Example usage:
# Replace Graph with your graph implementation
# graph = Graph()  # Assuming you have a graph instance
longest_path, length = longest_path_dijkstra(graph)
print(longest_path)
# print("Length:", length)


In [ ]:
# Block 1 - A star n300

graph = read_graph_from_file("graph_n300_r0.08202289062500001.EDGES")

import heapq

def initialize_single_source_max(graph, source, destination):
    for vertex in graph.vertices.values():
        vertex.d = float('-inf')
        vertex.h = euclidean_distance(vertex, destination)
        vertex.pi = None
        vertex.pi_set = set()

    source.d = 0

def relax_max(u, v):
    if v.d < (u.d + 1):
        v.d = u.d + 1
        v.pi = u
        return True
    return False

def a_star_max(graph, source, destination):
    initialize_single_source_max(graph, source, destination)
    S = set()  # Closed list
    Q = []  # Max heap
    for vertex in graph.vertices.values():
        vertex.priority = vertex.d + vertex.h
        heapq.heappush(Q, vertex)
    heapq._heapify_max(Q)

    while Q:
        u = heapq._heappop_max(Q)
        S.add(u)

        for v in graph.get_adjacency_list(u.key):
            v = graph.vertices[v]
            if v in u.pi_set:
                continue
            if relax_max(u, v):
                v.pi_set = u.pi_set | {u}
                if v in S:
                    S.remove(v)
                    heapq.heappush(Q, vertex)
                    heapq._heapify_max(Q)
                else:
                    v.priority = v.d + v.h
                    heapq._heapify_max(Q)

final_path = []

for i in range(0, len(graph.vertices)):
    if i in graph.vertices:
        for j in range(0, len(graph.vertices)):
            if i != j and j in graph.vertices:
                s = graph.vertices[i]
                d = graph.vertices[j]
                a_star_max(graph, s, d)
                path = []
                x = d
                if x.d != float('-inf'):
                    visited = {x}
                    while x != s:
                        path.insert(0, x.key)
                        visited.add(x)
                        x = x.pi
                        if x in visited:
                            break
                    path.insert(0, s.key)
                    if (len(path) > len(final_path)):
                        final_path = path

final_path

                

In [ ]:
# Block 2
# A star n400

graph = read_graph_from_file("graph_n400_r0.06249125.EDGES")

import heapq

def initialize_single_source_max(graph, source, destination):
    for vertex in graph.vertices.values():
        vertex.d = float('-inf')
        vertex.h = euclidean_distance(vertex, destination)
        vertex.pi = None
        vertex.pi_set = set()

    source.d = 0

def relax_max(u, v):
    if v.d < (u.d + 1):
        v.d = u.d + 1
        v.pi = u
        return True
    return False

def a_star_max(graph, source, destination):
    initialize_single_source_max(graph, source, destination)
    S = set()  # Closed list
    Q = []  # Max heap
    for vertex in graph.vertices.values():
        vertex.priority = vertex.d + vertex.h
        heapq.heappush(Q, vertex)
    heapq._heapify_max(Q)

    while Q:
        u = heapq._heappop_max(Q)
        S.add(u)

        for v in graph.get_adjacency_list(u.key):
            v = graph.vertices[v]
            if v in u.pi_set:
                continue
            if relax_max(u, v):
                v.pi_set = u.pi_set | {u}
                if v in S:
                    S.remove(v)
                    heapq.heappush(Q, vertex)
                    heapq._heapify_max(Q)
                else:
                    v.priority = v.d + v.h
                    heapq._heapify_max(Q)

final_path = []

for i in range(0, len(graph.vertices)):
    if i in graph.vertices:
        for j in range(0, len(graph.vertices)):
            if i != j and j in graph.vertices:
                s = graph.vertices[i]
                d = graph.vertices[j]
                a_star_max(graph, s, d)
                path = []
                x = d
                if x.d != float('-inf'):
                    visited = {x}
                    while x != s:
                        path.insert(0, x.key)
                        visited.add(x)
                        x = x.pi
                        if x in visited:
                            break
                    path.insert(0, s.key)
                    if (len(path) > len(final_path)):
                        final_path = path

final_path

                

In [ ]:
# Block 3
# A star n500

graph = read_graph_from_file("graph_n500_r0.05467859375.EDGES")

import heapq

def initialize_single_source_max(graph, source, destination):
    for vertex in graph.vertices.values():
        vertex.d = float('-inf')
        vertex.h = euclidean_distance(vertex, destination)
        vertex.pi = None
        vertex.pi_set = set()

    source.d = 0

def relax_max(u, v):
    if v.d < (u.d + 1):
        v.d = u.d + 1
        v.pi = u
        return True
    return False

def a_star_max(graph, source, destination):
    initialize_single_source_max(graph, source, destination)
    S = set()  # Closed list
    Q = []  # Max heap
    for vertex in graph.vertices.values():
        vertex.priority = vertex.d + vertex.h
        heapq.heappush(Q, vertex)
    heapq._heapify_max(Q)

    while Q:
        u = heapq._heappop_max(Q)
        S.add(u)

        for v in graph.get_adjacency_list(u.key):
            v = graph.vertices[v]
            if v in u.pi_set:
                continue
            if relax_max(u, v):
                v.pi_set = u.pi_set | {u}
                if v in S:
                    S.remove(v)
                    heapq.heappush(Q, vertex)
                    heapq._heapify_max(Q)
                else:
                    v.priority = v.d + v.h
                    heapq._heapify_max(Q)

final_path = []

for i in range(0, len(graph.vertices)):
    if i in graph.vertices:
        for j in range(0, len(graph.vertices)):
            if i != j and j in graph.vertices:
                s = graph.vertices[i]
                d = graph.vertices[j]
                a_star_max(graph, s, d)
                path = []
                x = d
                if x.d != float('-inf'):
                    visited = {x}
                    while x != s:
                        path.insert(0, x.key)
                        visited.add(x)
                        x = x.pi
                        if x in visited:
                            break
                    path.insert(0, s.key)
                    if (len(path) > len(final_path)):
                        final_path = path

final_path

                

In [ ]:
# Block 4
# DFS n300

graph = read_graph_from_file("graph_n300_r0.08202289062500001.EDGES")

def DFSAlgo(graph):
    def dfs(node, visited, path):
        nonlocal max_length, longest_path
        visited[node] = True
        path.append(node)

        for neighbor in graph.adjacency_list[node]:
            if not visited[neighbor]:
                dfs(neighbor, visited, path)

        if len(path) > max_length:
            max_length = len(path)
            longest_path = path.copy()

        path.pop()
        visited[node] = False

    max_length = float('-inf')
    longest_path = []

    visited = {v: False for v in graph.vertices}

    for vertex in graph.vertices:
        dfs(vertex, visited, [])

    return longest_path


DFSAlgo(graph)

In [ ]:
# Block 4
# DFS n400

graph = read_graph_from_file("graph_n400_r0.06249125.EDGES")

def DFSAlgo(graph):
    def dfs(node, visited, path):
        nonlocal max_length, longest_path
        visited[node] = True
        path.append(node)

        for neighbor in graph.adjacency_list[node]:
            if not visited[neighbor]:
                dfs(neighbor, visited, path)

        if len(path) > max_length:
            max_length = len(path)
            longest_path = path.copy()

        path.pop()
        visited[node] = False

    max_length = float('-inf')
    longest_path = []

    visited = {v: False for v in graph.vertices}

    for vertex in graph.vertices:
        dfs(vertex, visited, [])

    return longest_path


DFSAlgo(graph)

In [ ]:
# Block 4
# DFS n500

graph = read_graph_from_file("graph_n500_r0.05467859375.EDGES")

def DFSAlgo(graph):
    def dfs(node, visited, path):
        nonlocal max_length, longest_path
        visited[node] = True
        path.append(node)

        for neighbor in graph.adjacency_list[node]:
            if not visited[neighbor]:
                dfs(neighbor, visited, path)

        if len(path) > max_length:
            max_length = len(path)
            longest_path = path.copy()

        path.pop()
        visited[node] = False

    max_length = float('-inf')
    longest_path = []

    visited = {v: False for v in graph.vertices}

    for vertex in graph.vertices:
        dfs(vertex, visited, [])

    return longest_path


DFSAlgo(graph)